In [1]:
import opencc; cc = opencc.OpenCC('t2s.json')

from dtl import dtl_mongo; dbi = dtl_mongo.DTLMongo()
from sdemb import data

In [2]:
ccp_new_domains = [
    'qstheory.cn',
    'people.com.cn',
    'zgw-bj.com',
    'xinhua.net',
    'huanqiu.com',
    'hubpd.com',
]
tw_domains = [
    'ltn.com.tw',
    'chinatimes.com',
    
]
domains = ccp_new_domains + tw_domains

In [3]:
domain_map = {
    'ccp': ccp_new_domains,
    'liberty_times': ['ltn.com.tw'],
    'china_times': ['chinatimes.com'],
}

In [4]:
class DtlData(data.RawData):
    
    def __init__(self):
        super().__init__(
            corpus_name='dtl1',
            group_names=['ccp', 'liberty_times', 'china_times'],
            min_tok_count=5,
            n_vocab=20000)
    
    def doc_ids(self, group_name):
        doc_ids = []
        for domain in domain_map[group_name]:
            doc_ids += [str(x['articleId']) for x in dbi.jtoks.search(domain=domain)]
        return doc_ids
    
    def doc_tokens(self, doc_id):
        tokens = []
        article = dbi.jtoks.get(articleId=doc_id)
        for sent in article['sents']:
            tokens += sent
        if article['domain'] in tw_domains:
            tokens = [cc.convert(t) for t in tokens]
        return tokens

In [5]:
raw_data = DtlData()

In [6]:
corpus = data.Corpus.from_data(raw_data)

In [7]:
len(corpus.vocab)

13997